In [ ]:
from tqdm import tqdm
import pandas as pd
from konlpy.tag import Okt  # Okt  형태소 분석기 불러오기
import re
from collections import Counter

In [2]:
# Step 1: 데이터 로드
file_path = "C:/Users/cho03/Desktop/대학/4학년/2학기/캡스톤디자인-여행사이트/data/lodging_reviews.csv"
data = pd.read_csv(file_path)
data = data.replace("\n", " ", regex=True)  # 개행 문자를 공백으로 대체

# Step 2: 'review_text' 또는 'rating' 컬럼에 NaN이 있는 행 삭제
data = data.dropna(subset=["review_text", "rating"])

# 리뷰 텍스트의 길이를 계산하여 'length' 컬럼에 저장
data["length"] = data["review_text"].str.len()

In [3]:
# Step 3: 특수 문자 제거 (한글, 영어, 숫자만 유지)
data["review_text"] = data["review_text"].apply(lambda x: re.sub("[^0-9a-zA-Zㄱ-ㅣ가-힣 ]", "", str(x)))

In [4]:
# Step 4 : 의미없는 자음, 모음만 쓴 것 제거
def cleanText(readData):
    text = re.sub("[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]", "", readData)
    return text


# 'review_text' 컬럼에서 의미 없는 자음, 모음 제거
data["review_text"] = data["review_text"].apply(cleanText)

# 리뷰의 길이가 10 미만인 데이터 제거
data = data[data["length"] >= 10]
data

,lodging_id,rating,review_text,photos,date,length
0,1,5,아래 악플에 빡쳐서 리뷰 안남기는 사람인데 남깁니다 말그대로 이곳은 체험형 숙소다...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2021.11.14.,659
3,1,5,여기 너무 좋아요 또 가고 싶어요,NaN,2019.10.24.,20
4,2,3,공간 넓고 시설은 좋다 10만원 넘는 가격은 좀 세다고 본다,['https://img1.kakaocdn.net/cthumb/local/C139x...,2024.05.15.,37
6,2,1,다른 호텔로 예약해야 되는데 실수로 이 호텔로 예약하게 됨 그걸 알고 10분도 안지...,NaN,2024.01.12.,260
7,2,5,리모델링 전 모습은 모르겠으나 시설은 고충스러운 느낌을 줌 대한제국 갬성을 느낄수있...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2023.12.20.,70
...,...,...,...,...,...,...
46561,3822,5,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘됨,NaN,2023.03.26.,37
46563,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,311
46564,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요 침대도 커서 편해요 특히 조식 기대이상입...,NaN,2022.01.14.,68
46565,3822,5,사장님께서 정말 친절하시고 세심하게 챙겨주셨습니다 전자레인지 없는 방에 묶었는데 편...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2020.12.27.,183


In [5]:
# Step 5: 각 숙소(lodging_id)별 전체 리뷰를 기준으로 평균 평점과 리뷰 개수 계산
lodging_summary = (
    data.groupby("lodging_id")
    .agg(total_review_count=("rating", "count"), average_rating=("rating", "mean"))
    .reset_index()
)

# 조건 1: 전체 리뷰의 평균 평점이 4 이상인 숙소
qualified_lodging_ids = lodging_summary[lodging_summary["average_rating"] >= 4]["lodging_id"]


In [6]:
# Step 6: 조건을 만족하는 숙소 중 평점이 4 이상인 리뷰만 필터링
good_review = data[(data['lodging_id'].isin(qualified_lodging_ids)) & (data['rating'] >= 4)]
good_review

,lodging_id,rating,review_text,photos,date,length
0,1,5,아래 악플에 빡쳐서 리뷰 안남기는 사람인데 남깁니다 말그대로 이곳은 체험형 숙소다...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2021.11.14.,659
3,1,5,여기 너무 좋아요 또 가고 싶어요,NaN,2019.10.24.,20
20,3,5,공영주차장 가까운 편이고 방 깨끗하고 시원해요,NaN,2024.09.23.,25
21,3,5,방이 깨끗하고 사장님이 친절하세요,NaN,2019.11.24.,18
22,3,4,위치가 좋아 더운날씨에 잠깐씩 방에서 쉬다가 한옥마을 구경할수있어 편하게 이용하였어...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2019.08.11.,146
...,...,...,...,...,...,...
46559,3822,5,아주 작지만 있을건 다있는 정갈한곳이었다 아침조식은 12첩반상정도로 깔끔한 밥상이었...,NaN,2024.04.07.,78
46561,3822,5,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘됨,NaN,2023.03.26.,37
46563,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,311
46564,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요 침대도 커서 편해요 특히 조식 기대이상입...,NaN,2022.01.14.,68


In [7]:
# Step 7: 평점이 4 이상인 리뷰 개수가 5개 이상인 숙소만 선택
final_lodging_summary = (
    good_review.groupby("lodging_id")
    .agg(good_review_count=("rating", "count"))
    .reset_index()
)

In [8]:
# 평점 4 이상 리뷰가 5개 이상인 숙소 ID 추출
final_lodging_ids = final_lodging_summary[final_lodging_summary["good_review_count"] >= 5]["lodging_id"]

# Step 8
# : 최종 필터링 - 전체 조건을 만족하는 숙소의 평점 4 이상인 리뷰만 추출
filtered_data = good_review[good_review["lodging_id"].isin(final_lodging_ids)]

data = filtered_data

#### 불용어 처리


In [9]:
okt = Okt()

In [10]:
# 불용어 파일을 열어 내용 읽기
f = open("stopwords-ko.txt", "r", encoding="UTF-8")
st = f.readlines()  # 불용어 사전을 한 줄씩 읽어서 각 줄을 리스트의 요소로 저장
f.close()  # 파일 닫기

# 불용어 목록에서 줄바꿈 문자 제거
stw = []  # 줄바꿈 문자가 제거된 불용어를 저장할 리스트
for word in st:
    cleaned_word = word.strip()  # 각 단어에서 줄바꿈을 제거
    stw.append(cleaned_word)  # 제거된 단어를 stw 리스트에 추가

# 사용자가 추가한 불용어 리스트
user_stopwords = []  # 예시로 빈 리스트로 시작 (필요한 경우 여기에 단어 추가 가능)

# 불용어 목록을 사용자 불용어와 합친 뒤 중복 제거
stw.extend(user_stopwords)  # 기존 불용어와 사용자 불용어를 합침
stopwords = list(set(stw))  # 중복 제거 후 최종 불용어 리스트 생성

In [11]:
# Step 5: 다양한 품사 추출 (명사 외에도 형용사, 동사 등)
tokenized_text = []  # 각 리뷰에서 추출한 형태소를 담을 리스트

for i in tqdm(range(len(data))):
    review_text = data["review_text"].iloc[i]  # 현재 리뷰의 텍스트 가져오기
    tokens = []  # 현재 리뷰에서 추출한 단어들을 담을 리스트

    # 형태소 분석기를 사용해 단어와 품사 추출
    for word, pos in okt.pos(review_text):
        # 품사가 명사, 형용사, 또는 동사인 경우에만 추가
        if pos in ["Noun", "ProperNoun", "Adjective", "Verb", "VerbPrefix"]:
            tokens.append(word)

    # 현재 리뷰의 결과를 전체 리스트에 추가
    tokenized_text.append(tokens)

100%|██████████| 8789/8789 [00:48<00:00, 182.87it/s]


In [12]:
def remove_stopwords(tokenized_reviews, stopwords):
    cleaned_reviews = []  # 불용어가 제거된 리뷰를 담을 리스트

    # 불용어 제거 과정의 진행 상황을 시각화
    for tokens in tqdm(tokenized_reviews, desc="불용어 처리 중"):
        # 각 리뷰에서 불용어 제거한 결과를 담을 리스트
        cleaned_tokens = []

        # 리뷰 내의 각 단어를 검사
        for word in tokens:
            # 단어가 불용어 목록에 없으면 cleaned_tokens 리스트에 추가
            if word not in stopwords:
                cleaned_tokens.append(word)

        # 현재 리뷰에서 불용어가 제거된 결과를 cleaned_reviews 리스트에 추가
        cleaned_reviews.append(cleaned_tokens)

    return cleaned_reviews  # 불용어가 제거된 리뷰 리스트 반환

In [13]:
# Step 7: 불용어 제거 및 결과 저장
data["review_after"] = remove_stopwords(tokenized_text, stopwords)

불용어 처리 중: 100%|██████████| 8789/8789 [00:01<00:00, 5318.88it/s]
C:\Users\cho03\AppData\Local\Temp\ipykernel_8640\2410181585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["review_after"] = remove_stopwords(tokenized_text, stopwords)


In [ ]:
from sentence_transformers import SentenceTransformer

# KoBERT 모델 로드
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 태그별 예시 문장 정의
tag_examples = {
    "가성비": [
        "가격 대비 만족스러운 숙소입니다.",
        "비용에 비해 기대 이상이었어요.",
        "저렴한 가격에 만족스러웠습니다.",
        "합리적인 가격으로 숙박하기 좋았습니다.",
        "가격이 저렴하면서 필요한 시설이 잘 갖춰져 있어요.",
        "가성비가 좋아요.",
        "가격 대비 품질이 좋습니다.",
        "가격대비 만족합니다.",
        "비용에 비해 적절한 선택이었어요.",
        "가격 부담 없이 즐길 수 있는 숙소였습니다.",
    ],
    "청결": [
        "객실이 매우 깨끗하게 유지되고 있었습니다.",
        "전체적으로 깔끔하게 관리되고 있어요.",
        "청결 상태가 좋아서 안심이 됩니다.",
        "방과 욕실 모두 깨끗했습니다.",
        "숙소가 위생적으로 관리되고 있어요.",
        "깔끔하고 청결한 공간이었습니다.",
        "정돈이 잘 되어 있어요.",
        "청결도가 좋아 만족했습니다.",
        "깨끗하고 위생적입니다.",
        "숙소가 깔끔하게 관리되고 있습니다.",
        "위생 관리가 철저한 느낌이었습니다.",
        "쾌적한 환경을 유지하는 숙소였습니다.",
    ],
    "직원 만족": [
        "직원들이 정말 친절하게 응대해주셨어요.",
        "서비스가 매우 세심하고 좋았습니다.",
        "프론트 직원이 특히 친절했어요.",
        "요청 사항에 빠르게 대응해 주셨습니다.",
        "직원들의 서비스 마인드가 훌륭합니다.",
        "친절하고 상냥한 직원들이 있습니다.",
        "직원들이 매우 협조적입니다.",
        "친절한 서비스 덕분에 편안하게 머물렀습니다.",
        "직원 서비스가 인상적이었어요.",
        "서비스 수준이 높습니다.",
    ],
    "위치": [
        "위치가 좋아 이동하기 편리했습니다.",
        "관광지와 가까워서 좋았어요.",
        "대중교통과 접근성이 좋습니다.",
        "중심지에 위치해 주변 시설 이용이 편리했습니다.",
        "조용한 곳에 위치해 휴식하기 좋았어요.",
        "교통이 편리해요.",
        "중심지와 가까워요.",
        "위치가 좋아서 접근성이 좋습니다.",
        "이동하기에 매우 편리합니다.",
        "주변 시설 접근성이 훌륭해요.",
    ],
    "가족 여행": [
        "아이들과 머물기에 좋은 숙소입니다.",
        "가족 단위로 이용하기에 편리해요.",
        "가족 여행에 최적화된 시설입니다.",
        "가족이 함께 즐길 수 있는 공간이 있어 좋았습니다.",
        "아이들이 즐길 수 있는 편의 시설이 잘 갖춰져 있어요.",
        "가족 여행에 좋습니다.",
        "가족끼리 오기 좋았어요.",
        "아이들과 함께 지내기 좋아요.",
        "아이들과 함께 이용하기에 안성맞춤입니다.",
    ],
    "연인": [
        "로맨틱한 분위기로 커플 여행에 좋았습니다.",
        "연인과 함께하기에 이상적인 숙소예요.",
        "특별한 날에 방문하기 좋은 곳입니다.",
        "아늑한 분위기가 좋아 연인과 추천합니다.",
        "연인과 특별한 시간을 보내기 좋았습니다.",
        "커플에게 딱 맞는 숙소예요.",
        "로맨틱한 분위기가 좋았어요.",
        "연인과 함께할 분위기였습니다.",
        "둘만의 특별한 시간을 보내기 좋은 곳입니다.",
    ],
    "풍경": [
        "자연 경관이 멋진 숙소입니다.",
        "탁 트인 전망이 인상적이에요.",
        "숙소 주변의 풍경이 아름다웠습니다.",
        "뷰가 좋아 마음이 편안해졌어요.",
        "산과 호수의 멋진 뷰를 즐길 수 있어 좋았어요.",
        "풍경이 아름다워요.",
        "경치가 좋아서 힐링됩니다.",
        "멋진 전망을 감상할 수 있어요.",
        "뷰가 정말 인상 깊었습니다.",
    ],
}

c:\python-capstone\review_tag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [15]:
# 태그 예시 문장을 KoBERT로 임베딩 생성
tag_embeddings = {}  # 각 태그의 임베딩을 저장할 딕셔너리

# 태그와 해당 문장 예시를 순회
for tag, sentences in tqdm(tag_examples.items(), desc="태그 예시 문장 임베딩 생성 중"):
    # 해당 태그의 문장 예시를 모델을 사용해 임베딩으로 변환
    embedding = model.encode(sentences)
    print(embedding)

    # 변환된 임베딩을 태그 이름과 함께 딕셔너리에 저장
    tag_embeddings[tag] = embedding

태그 예시 문장 임베딩 생성 중:  33%|███▎      | 2/6 [00:01<00:03,  1.26it/s]

[[-1.3016224  -0.02157879 -0.4333049  ... -0.9648594   0.319554
  -0.44159606]
 [-0.60578465  0.6309073  -0.69466954 ... -0.08758131 -0.08981818
   0.82778573]
 [-1.2027721  -0.36033323 -0.9524694  ... -0.72653097  0.48771006
   0.52121407]
 [-1.01127    -0.40440172 -0.21071719 ... -0.88906264  0.6536452
   0.4188936 ]
 [-0.82521224  0.26828083  0.1878172  ... -0.9886759   0.62298554
  -0.8337744 ]]
[[-0.7569053  -0.02862204 -0.31508693 ... -0.07689787 -0.59673846
  -0.11341874]
 [-1.0775527  -0.19315977 -0.408174   ...  0.32238728 -0.03507136
  -0.47158772]
 [-0.24677446 -0.49605528 -0.91742957 ...  0.00424872  0.06680536
  -0.25089508]
 [-0.01952896 -0.8667161  -0.17021014 ...  0.31691313 -0.7911758
  -0.07460344]
 [-0.8818594  -0.30252695  0.52037936 ... -0.06234384 -0.7865844
  -0.68696296]]


태그 예시 문장 임베딩 생성 중:  67%|██████▋   | 4/6 [00:02<00:00,  2.71it/s]

[[-0.8983776  -0.35127914  0.01862473 ... -0.5414077  -0.03152392
   0.4643566 ]
 [-1.2826736  -0.83193904 -1.2821002  ... -0.13691524  0.14821723
   0.61640203]
 [-0.7977897  -0.79110277 -0.68951887 ... -0.03246104  0.23152229
  -0.1070595 ]
 [-0.6835452  -0.64987725 -0.1734     ... -0.60988086 -0.17362818
   0.30601048]
 [-1.013245    0.10852399 -0.84956485 ... -0.19133772  0.07397249
  -0.3531901 ]]
[[ 0.04721453 -1.2803679  -0.406258   ... -0.5836866  -0.14091963
  -0.780996  ]
 [-0.04430345 -1.0000327  -1.3764075  ... -0.9299524  -0.11928582
  -0.3184767 ]
 [-0.04393333 -1.1463977  -0.66373664 ... -0.7849408  -0.30726528
  -0.42009684]
 [-0.7067266  -1.1823659  -0.53304917 ... -0.705096    0.22855611
  -0.04949956]
 [-0.19561414 -1.7501599  -0.52566767 ...  0.12308456 -0.6460303
  -0.02984549]]


태그 예시 문장 임베딩 생성 중: 100%|██████████| 6/6 [00:02<00:00,  2.35it/s]

[[-0.63167477 -0.15118472 -0.69939685 ... -0.01834917 -0.22626585
  -0.41320822]
 [-1.0566225  -1.466212   -0.50075513 ...  0.5518383   0.70424473
   0.59996146]
 [-1.2821057  -1.1098505   0.3506238  ...  0.11931556 -0.09817334
  -0.29876968]
 [-0.2762928  -1.0962316  -0.97901076 ...  0.2518254  -0.24577993
   0.560846  ]
 [-0.63330674 -0.3901284  -0.09374593 ... -0.5964876   0.50462997
  -0.2553251 ]]
[[-0.75509316 -1.2674809  -0.3929348  ... -0.49799046  0.51896095
   0.07652872]
 [-1.085387   -0.8416821   0.19270109 ... -0.01060676  0.1639307
  -1.2048237 ]
 [-0.33940092 -0.07183681 -0.6736004  ... -0.54218733  0.6781858
   0.00374659]
 [-1.2250346  -1.1310663  -0.16755223 ...  0.23265529  0.95856977
  -0.18370922]
 [-0.28883243 -0.8949187  -0.85501766 ... -0.744165    0.34321463
  -0.07932944]]


In [16]:
# 리뷰 문장 임베딩 생성
review_embeddings = []  # 각 리뷰의 임베딩을 저장할 리스트

# tqdm을 사용하여 각 리뷰에 대해 임베딩 생성 과정을 시각화
for tokens in tqdm(data["review_after"], desc="리뷰 임베딩 생성 중"):
    # 리스트 형태의 토큰을 하나의 문자열로 결합
    review_text = " ".join(tokens)

    # 모델을 사용하여 결합된 문자열을 임베딩으로 변환
    embedding = model.encode(review_text)

    # 변환된 임베딩을 review_embeddings 리스트에 추가
    review_embeddings.append(embedding)

리뷰 임베딩 생성 중: 100%|██████████| 8789/8789 [18:29<00:00,  7.92it/s]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 유사도 임계값 설정 (예시: 0.5)
threshold = 0.5

# 각 리뷰별 할당된 태그를 저장할 리스트
assigned_tags_per_review = []

# 각 리뷰의 임베딩을 기반으로 태그를 할당
for review_embedding in tqdm(review_embeddings, desc="리뷰별 태그 할당 중"):
    tag_scores = {}  # 각 태그에 대한 유사도 점수를 저장할 딕셔너리

    # 각 태그의 임베딩과 현재 리뷰의 유사도를 계산
    for tag, embeddings in tag_embeddings.items():
        # 리뷰 임베딩과 태그 예시 임베딩들 간의 코사인 유사도를 계산
        similarities = cosine_similarity([review_embedding], embeddings)

        # 유사도의 평균값을 현재 태그의 유사도 점수로 저장
        tag_scores[tag] = similarities.mean()

    # 유사도 점수가 임계값(threshold) 이상인 태그를 선택
    best_tags = []
    for tag, score in tag_scores.items():
        if score > threshold:
            best_tags.append(tag)

    # 현재 리뷰에 할당된 태그 리스트를 전체 리스트에 추가
    assigned_tags_per_review.append(best_tags)

# 할당된 태그를 데이터프레임에 추가
data["assigned_tags"] = assigned_tags_per_review

리뷰별 태그 할당 중: 100%|██████████| 8789/8789 [00:34<00:00, 251.24it/s]
C:\Users\cho03\AppData\Local\Temp\ipykernel_8640\1566289521.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["assigned_tags"] = assigned_tags_per_review


In [18]:
# 리뷰별 태그 빈도 계산 (풀어서 작성)
review_tag_counts = []
for tags in tqdm(filtered_data["assigned_tags"], desc="리뷰별 태그 및 빈도 계산"):
    tag_counts = Counter(tags)  # 각 리뷰의 태그 빈도 계산
    review_tag_counts.append(
        {
            "리뷰의_가성비": tag_counts.get("가성비", 0),
            "리뷰의_청결도": tag_counts.get("청결", 0),
            "리뷰의_직원_만족": tag_counts.get("직원 만족", 0),
            "리뷰의_위치": tag_counts.get("위치", 0),
            "리뷰의_가족_여행": tag_counts.get("가족 여행", 0),
            "리뷰의_연인": tag_counts.get("연인", 0),
            "리뷰의_풍경": tag_counts.get("풍경", 0),
        }
    )

# 리뷰별 태그 빈도 리스트를 데이터프레임으로 변환
review_tag_counts_df = pd.DataFrame(review_tag_counts)

# 리뷰 데이터에 리뷰별 태그 빈도 추가
final_review_data = pd.concat([filtered_data.reset_index(drop=True), review_tag_counts_df], axis=1)
final_review_data

리뷰별 태그 및 빈도 계산: 100%|██████████| 8789/8789 [00:00<00:00, 297386.54it/s]

,lodging_id,rating,review_text,photos,date,length,review_after,assigned_tags,리뷰의_가성비,리뷰의_청결도,리뷰의_직원_만족,리뷰의_위치,리뷰의_가족_여행,리뷰의_연인,리뷰의_풍경
0,6,5,깨끗하고 깔끔합니다 제주일주 하면서 이렇게 깨끗한곳은 처음이네요 방충망도 꼼꼼하게 ...,NaN,2024.10.25.,177,"[깨끗하고, 깔끔합니다, 제주, 일주, 깨끗한, 곳, 처음, 요, 방충, 망, 꼼꼼...",[],0,0,0,0,0,0,0
1,6,5,제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다 시...,NaN,2023.02.17.,161,"[게스트하우스, 가보, 않았지만, 정말, 최고, 숙소, 할, 수, 있습니다, 시설,...","[가성비, 가족 여행]",1,0,0,0,1,0,0
2,6,4,뷰가 일단 대박이구요 룸 컨디션은 쏘쏘했는데 세탁실 쓸수있고 공용 라운지에서 조식먹...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.10.01.,71,"[뷰, 대박, 이구, 룸, 컨디션, 쏘, 쏘, 했는데, 세탁실, 쓸수있고, 공용, ...",[],0,0,0,0,0,0,0
3,6,5,제주에 오면 꼭 다시 오고 싶은 곳넓고 깔끔 쾌적세탁기실도 갖춰 편리편백나무 향나는...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.07.02.,189,"[제주, 오면, 꼭, 다시, 오고, 싶은, 곳, 넓고, 깔, 끔, 쾌적, 세탁기, ...",[],0,0,0,0,0,0,0
4,6,5,사장님의 세심함이 느껴지는 게하 서울부터 전국여행 중인 부부입니다 수 많은 게하나 ...,NaN,2022.06.26.,386,"[사장, 세심, 함, 느껴지는, 게, 서울, 전국, 여행, 중인, 부부, 입니다, ...",[],0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,3822,5,아주 작지만 있을건 다있는 정갈한곳이었다 아침조식은 12첩반상정도로 깔끔한 밥상이었...,NaN,2024.04.07.,78,"[아주, 작, 있을건, 있는, 정갈한, 곳, 아침, 조식, 첩, 반상, 정도, 깔끔...",[],0,0,0,0,0,0,0
8785,3822,5,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘됨,NaN,2023.03.26.,37,"[찐, 짜, 청결함, 방, 작, 바닥, 따뜻하고, 물, 온도, 조절, 잘, 됨]",[],0,0,0,0,0,0,0
8786,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,311,"[덕유산, 등산, 하기, 전, 박한, 숙소, 입니다, 긴박하게, 잡아, 선택, 여지...",[],0,0,0,0,0,0,0
8787,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요 침대도 커서 편해요 특히 조식 기대이상입...,NaN,2022.01.14.,68,"[건물, 시설, 청결하고, 방이, 무척, 따끈해요, 침대, 커서, 편해요, 조식, ...",[],0,0,0,0,0,0,0


In [19]:
# 숙소별 태그 빈도 계산 및 상위 2개 태그 선정
final_tags_per_lodging = []
lodging_ids = final_review_data["lodging_id"].unique()

for lodging_id in tqdm(lodging_ids, desc="숙소별 태그 및 빈도 계산"):
    keywords_all_reviews = []
    for tags in final_review_data[final_review_data["lodging_id"] == lodging_id]["assigned_tags"]:
        keywords_all_reviews.extend(tags)

    # 태그 빈도 계산
    tag_counts = Counter(keywords_all_reviews)
    top_2_tags = []

    # 상위 2개의 태그 중 빈도가 3 이상인 경우만 추가
    for tag, count in tag_counts.most_common():
        if count >= 3:  # 빈도 조건을 충족하는지 확인
            top_2_tags.append(tag)
        if len(top_2_tags) == 2:  # 상위 2개 태그가 채워지면 종료
            break

    # 각 숙소의 태그 빈도 및 상위 태그 추가
    final_tags_per_lodging.append(
        {
            "lodging_id": lodging_id,
            "숙소의_가성비": tag_counts.get("가성비", 0),
            "숙소의_청결도": tag_counts.get("청결", 0),
            "숙소의_직원_만족": tag_counts.get("직원 만족", 0),
            "숙소의_위치": tag_counts.get("위치", 0),
            "숙소의_가족_여행": tag_counts.get("가족 여행", 0),
            "숙소의_연인": tag_counts.get("연인", 0),
            "숙소의_풍경": tag_counts.get("풍경", 0),
            "숙소의_top_2_tags": top_2_tags,
        }
    )

숙소별 태그 및 빈도 계산: 100%|██████████| 421/421 [00:00<00:00, 1044.90it/s]


In [20]:
# 최종 숙소별 태그 빈도를 데이터프레임으로 변환하고 원래 데이터와 병합
lodging_tags_specified_df = pd.DataFrame(final_tags_per_lodging)
final_data = pd.merge(final_review_data, lodging_tags_specified_df, on="lodging_id", how="left")

In [21]:
final_data

,lodging_id,rating,review_text,photos,date,length,review_after,assigned_tags,리뷰의_가성비,리뷰의_청결도,...,리뷰의_연인,리뷰의_풍경,숙소의_가성비,숙소의_청결도,숙소의_직원_만족,숙소의_위치,숙소의_가족_여행,숙소의_연인,숙소의_풍경,숙소의_top_2_tags
0,6,5,깨끗하고 깔끔합니다 제주일주 하면서 이렇게 깨끗한곳은 처음이네요 방충망도 꼼꼼하게 ...,NaN,2024.10.25.,177,"[깨끗하고, 깔끔합니다, 제주, 일주, 깨끗한, 곳, 처음, 요, 방충, 망, 꼼꼼...",[],0,0,...,0,0,2,1,2,0,1,0,0,[]
1,6,5,제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다 시...,NaN,2023.02.17.,161,"[게스트하우스, 가보, 않았지만, 정말, 최고, 숙소, 할, 수, 있습니다, 시설,...","[가성비, 가족 여행]",1,0,...,0,0,2,1,2,0,1,0,0,[]
2,6,4,뷰가 일단 대박이구요 룸 컨디션은 쏘쏘했는데 세탁실 쓸수있고 공용 라운지에서 조식먹...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.10.01.,71,"[뷰, 대박, 이구, 룸, 컨디션, 쏘, 쏘, 했는데, 세탁실, 쓸수있고, 공용, ...",[],0,0,...,0,0,2,1,2,0,1,0,0,[]
3,6,5,제주에 오면 꼭 다시 오고 싶은 곳넓고 깔끔 쾌적세탁기실도 갖춰 편리편백나무 향나는...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.07.02.,189,"[제주, 오면, 꼭, 다시, 오고, 싶은, 곳, 넓고, 깔, 끔, 쾌적, 세탁기, ...",[],0,0,...,0,0,2,1,2,0,1,0,0,[]
4,6,5,사장님의 세심함이 느껴지는 게하 서울부터 전국여행 중인 부부입니다 수 많은 게하나 ...,NaN,2022.06.26.,386,"[사장, 세심, 함, 느껴지는, 게, 서울, 전국, 여행, 중인, 부부, 입니다, ...",[],0,0,...,0,0,2,1,2,0,1,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,3822,5,아주 작지만 있을건 다있는 정갈한곳이었다 아침조식은 12첩반상정도로 깔끔한 밥상이었...,NaN,2024.04.07.,78,"[아주, 작, 있을건, 있는, 정갈한, 곳, 아침, 조식, 첩, 반상, 정도, 깔끔...",[],0,0,...,0,0,0,0,0,0,0,0,0,[]
8785,3822,5,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘됨,NaN,2023.03.26.,37,"[찐, 짜, 청결함, 방, 작, 바닥, 따뜻하고, 물, 온도, 조절, 잘, 됨]",[],0,0,...,0,0,0,0,0,0,0,0,0,[]
8786,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,311,"[덕유산, 등산, 하기, 전, 박한, 숙소, 입니다, 긴박하게, 잡아, 선택, 여지...",[],0,0,...,0,0,0,0,0,0,0,0,0,[]
8787,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요 침대도 커서 편해요 특히 조식 기대이상입...,NaN,2022.01.14.,68,"[건물, 시설, 청결하고, 방이, 무척, 따끈해요, 침대, 커서, 편해요, 조식, ...",[],0,0,...,0,0,0,0,0,0,0,0,0,[]


In [22]:
final_data.to_csv('review_tags(okt).csv', encoding='utf-8-sig', index=False)